# <center>Movie Based Recommender System<center>

# Import Libraries

In [1]:
#Making necesarry imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances
import ipywidgets as widgets
from ipywidgets import * 
import pickle
import nltk
nltk.download('punkt')
from IPython.display import display, clear_output
from contextlib import contextmanager
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os, sys
import re
import seaborn as sns

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


# Popularity Based Recommender

## Load Dataset

In [2]:
metadata = pd.read_csv('RecomSystm/data/movies_metadata.csv', low_memory=False)

# Print the first five rows
metadata.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## Data understanding and preprocessing

Lets calculate the total number of rows and columns present in dataset

In [3]:
metadata.shape

(45466, 24)

In [4]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

## Based on Number of Voters

In [5]:
### Feature Selection 

In [6]:
#Create a smaller dataframe with a subset of all features
small_df1 = metadata[['title', 'vote_count']]

#Output only the first 5 rows of small_df
small_df1.head()


,title,vote_count
0,Toy Story,5415.0
1,Jumanji,2413.0
2,Grumpier Old Men,92.0
3,Waiting to Exhale,34.0
4,Father of the Bride Part II,173.0


 **.tail()** method is use to display last five rows of dataframe.

In [7]:
small_df1.tail()

,title,vote_count
45461,Subdue,1.0
45462,Century of Birthing,3.0
45463,Betrayal,6.0
45464,Satan Triumphant,0.0
45465,Queerama,0.0


Lets see the details of small_df dataset using **.info()** method

In [8]:
small_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       45460 non-null  object 
 1   vote_count  45460 non-null  float64
dtypes: float64(1), object(1)
memory usage: 710.5+ KB


In [9]:
small_df1['vote_count'].isnull().values.any()


True

In [10]:
small_df1['vote_count'].isnull().sum()

6

In [11]:
small_df1 = small_df1.dropna(subset=['vote_count'])
small_df1.head()

,title,vote_count
0,Toy Story,5415.0
1,Jumanji,2413.0
2,Grumpier Old Men,92.0
3,Waiting to Exhale,34.0
4,Father of the Bride Part II,173.0


In [12]:
small_df1['vote_count'].isnull().sum()

0

In [13]:
#Convert vote_count to int
small_df1['vote_count']=small_df1['vote_count'].astype('int')


In [14]:
small_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45460 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       45460 non-null  object
 1   vote_count  45460 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 887.9+ KB


In [15]:
small_df1.tail()

,title,vote_count
45461,Subdue,1
45462,Century of Birthing,3
45463,Betrayal,6
45464,Satan Triumphant,0
45465,Queerama,0


In [16]:
small_df1.loc[small_df1['vote_count'] == 0]


,title,vote_count
83,Last Summer in the Hamptons,0
107,Headless Body in Topless Bar,0
126,Jupiter's Wife,0
132,Sonic Outlaws,0
137,Target,0
...,...,...
45432,Altar of Fire,0
45434,The Wonders of Aladdin,0
45452,Deep Hearts,0
45464,Satan Triumphant,0


In [17]:
small_df1.loc[small_df1['vote_count'] == 0] =np.nan

In [18]:
small_df1 = small_df1.dropna(subset=['vote_count'])


In [19]:
small_df1.tail()

,title,vote_count
45459,Caged Heat 3000,1.0
45460,Robin Hood,26.0
45461,Subdue,1.0
45462,Century of Birthing,3.0
45463,Betrayal,6.0


## Processing

After data pre-processing now we are ready to give recommendation of movies which have gained maximum votes.

In [20]:
# Output only the first 5 rows of small_df
small_df1=small_df1.sort_values('vote_count', ascending=False)
small_df1.head()

popular1 = small_df1.title
popular1=popular1.reset_index(drop=True)

small_df1.head()

,title,vote_count
15480,Inception,14075.0
12481,The Dark Knight,12269.0
14551,Avatar,12114.0
17818,The Avengers,12000.0
26564,Deadpool,11444.0


In [21]:
del small_df1['vote_count']


In [22]:
popular1 = small_df1
popular1=popular1.reset_index(drop=True)

popular1.head()

,title
0,Inception
1,The Dark Knight
2,Avatar
3,The Avengers
4,Deadpool


## Based on top Rated movies

In [23]:
#Create a smaller dataframe with a subset of all features
small_df2 = metadata[['title', 'vote_average']]

#Output only the first 5 rows of small_df
small_df2.head()


,title,vote_average
0,Toy Story,7.7
1,Jumanji,6.9
2,Grumpier Old Men,6.5
3,Waiting to Exhale,6.1
4,Father of the Bride Part II,5.7


In [24]:
small_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         45460 non-null  object 
 1   vote_average  45460 non-null  float64
dtypes: float64(1), object(1)
memory usage: 710.5+ KB


In [25]:
small_df2['vote_average'].isnull().sum()

6

In [26]:
small_df2 = small_df2.dropna(subset=['vote_average'])


In [27]:
small_df2['vote_average'].isnull().sum()

0

In [28]:
small_df2.head()

,title,vote_average
0,Toy Story,7.7
1,Jumanji,6.9
2,Grumpier Old Men,6.5
3,Waiting to Exhale,6.1
4,Father of the Bride Part II,5.7


## Processing

After data pre-processing now we are ready to give recommendations of top rated movies. 

In [29]:
# Output only the first 5 rows of small_df2
small_df2=small_df2.sort_values('vote_average', ascending=False)
small_df2.head()


,title,vote_average
21642,Ice Age Columbus: Who Were the First Americans?,10.0
15710,If God Is Willing and da Creek Don't Rise,10.0
22396,Meat the Truth,10.0
22395,Marvin Hamlisch: What He Did For Love,10.0
35343,Elaine Stritch: At Liberty,10.0


In [30]:
del small_df2['vote_average']
ranking = small_df2
ranking=ranking.reset_index(drop=True)
ranking.head(10)

,title
0,Ice Age Columbus: Who Were the First Americans?
1,If God Is Willing and da Creek Don't Rise
2,Meat the Truth
3,Marvin Hamlisch: What He Did For Love
4,Elaine Stritch: At Liberty
5,Reckless
6,The Human Surge
7,The Guide
8,هیچ کجا هیچ کس
9,Other Voices Other Rooms


## Limitations of Recommender

In the above two recommender systems there are some problems which may you have noticed. If not lets see below:
    
## 1. Based on top Rated movies

- On a related note, this metric will also tend to favor movies with smaller number of voters with skewed and/or extremely high ratings. As the number of voters increase, the rating of a movie regularizes and approaches towards a value that is reflective of the movie's quality. It is more difficult to discern the quality of a movie with extremely few voters.


- In small_df1 lets take one addition attribute vote_average to observe this drawback:

In [31]:
small_df1 = metadata[['title', 'vote_average', 'vote_count', ]]

# Output only the first 15 rows of small_df1
small_df1=small_df1.sort_values('vote_average', ascending=False)
small_df1.head()

,title,vote_average,vote_count
21642,Ice Age Columbus: Who Were the First Americans?,10.0,1.0
15710,If God Is Willing and da Creek Don't Rise,10.0,1.0
22396,Meat the Truth,10.0,1.0
22395,Marvin Hamlisch: What He Did For Love,10.0,1.0
35343,Elaine Stritch: At Liberty,10.0,1.0


## Based Weighted Rating formula

In [32]:
#Create a smaller dataframe with a subset of all features
small_df3 = metadata[['title', 'vote_average','vote_count']]

#Output only the first 5 rows of small_df3
small_df3.head()


,title,vote_average,vote_count
0,Toy Story,7.7,5415.0
1,Jumanji,6.9,2413.0
2,Grumpier Old Men,6.5,92.0
3,Waiting to Exhale,6.1,34.0
4,Father of the Bride Part II,5.7,173.0


In [33]:
small_df3.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         45460 non-null  object 
 1   vote_average  45460 non-null  float64
 2   vote_count    45460 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.0+ MB


In [34]:
small_df3.loc[(small_df3['vote_average']==0) | (small_df3['vote_count']==0)] 


,title,vote_average,vote_count
83,Last Summer in the Hamptons,0.0,0.0
107,Headless Body in Topless Bar,0.0,0.0
126,Jupiter's Wife,0.0,0.0
132,Sonic Outlaws,0.0,0.0
137,Target,0.0,0.0
...,...,...,...
45432,Altar of Fire,0.0,0.0
45434,The Wonders of Aladdin,0.0,0.0
45452,Deep Hearts,0.0,0.0
45464,Satan Triumphant,0.0,0.0


In [35]:
small_df3.loc[(small_df3['vote_count']==0) | (small_df3['vote_average']==0)] = np.nan


In [36]:
small_df3.isnull().sum()

title           3004
vote_average    3004
vote_count      3004
dtype: int64

In [37]:
small_df3=small_df3.dropna()


In [38]:
small_df3.isnull().sum()

title           0
vote_average    0
vote_count      0
dtype: int64

In [39]:
#Convert vote_count to int
small_df3['vote_count']=small_df3['vote_count'].astype('int')


In [40]:
small_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42462 entries, 0 to 45463
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         42462 non-null  object 
 1   vote_average  42462 non-null  float64
 2   vote_count    42462 non-null  int32  
dtypes: float64(1), int32(1), object(1)
memory usage: 1.1+ MB


In [41]:
small_df3.head()

,title,vote_average,vote_count
0,Toy Story,7.7,5415
1,Jumanji,6.9,2413
2,Grumpier Old Men,6.5,92
3,Waiting to Exhale,6.1,34
4,Father of the Bride Part II,5.7,173


# Processing 

### Weighted rating formula calculation:

**Weighted Rating (WR) = `(v/v+m.R)+(m/v+m.C)`**

where,

- **v** is the number of votes for the movie;
- **m** is the minimum votes required to be listed in the chart;
- **R** is the average rating of the movie; And
- **C** is the mean vote across the whole report

We already have the values to **v (vote_count)** and **R (vote_average)** for each movie in the dataset. It is also possible to directly calculate **C** from this data.

In [42]:
# Calculate C
C = small_df3['vote_average'].mean()
C

6.014876831048623

In [43]:
# Calculate the minimum number of votes required to be in the chart, m
m = small_df3['vote_count'].quantile(0.80)
print(m)

56.0


In [44]:
# Filter out all qualified movies into a new DataFrame
q_movies = small_df3.copy().loc[small_df3['vote_count'] >= m]
q_movies.shape

(8561, 3)

## Calculate the score for every movie:

In [45]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m)   * R) + (m/(m+v) * C)

In [46]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [47]:
q_movies = q_movies.sort_values('score', ascending=False)
q_movies.head()

,title,vote_average,vote_count,score
10309,Dilwale Dulhania Le Jayenge,9.1,661,8.859042
314,The Shawshank Redemption,8.5,8358,8.483460
834,The Godfather,8.5,6024,8.477111
40251,Your Name.,8.5,1030,8.371854
12481,The Dark Knight,8.3,12269,8.289617


In [48]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)


In [49]:
imdb=q_movies[['title', 'vote_count', 'vote_average', 'score']].head(250)


In [50]:
del imdb['vote_count']
del imdb['score']
del imdb['vote_average']


In [51]:
#Print the top 250 movies

imdb_movie=imdb
imdb_movie=imdb_movie.reset_index(drop=True)
imdb_movie.head()

,title
0,Dilwale Dulhania Le Jayenge
1,The Shawshank Redemption
2,The Godfather
3,Your Name.
4,The Dark Knight


# The knowledge-based recommender

In [52]:
df = pd.read_csv('RecomSystm/data/movies_metadata.csv')


In [53]:
#Only keep those features that we require 
df = df[['title','genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]

df.head()

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [54]:
#Convert release_date into pandas datetime format
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

#Extract year from the datetime
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [55]:
df.head()

,title,genres,release_date,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0,1995


In [56]:
#Helper function to convert NaT to 0 and all other years to integers.
def convert_int(x):
    try:
        return int(x)
    except:
        return 0

In [57]:
#Apply convert_int to the year feature
df['year'] = df['year'].apply(convert_int)

In [58]:
#Drop the release_date column
df = df.drop('release_date', axis=1)

#Display the dataframe
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995


## Genres

In [59]:
#Print genres of the first movie
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [60]:
#Import the literal_eval function from ast
from ast import literal_eval

#Define a stringified list and output its type
a = "[1,2,3]"
print(type(a))

#Apply literal_eval and output type
b = literal_eval(a)
print(type(b))

<class 'str'>
<class 'list'>


In [61]:
#Convert all NaN into stringified empty lists
df['genres'] = df['genres'].fillna('[]')

#Apply literal_eval to convert stringified empty lists to the list object
df['genres'] = df['genres'].apply(literal_eval)

#Convert list of dictionaries to a list of strings
df['genres'] = df['genres'].apply(lambda x: [i['name'].lower() for i in x] if isinstance(x, list) else [])

In [62]:
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[comedy, drama, romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,1995


In [ ]:
#Create a new feature by exploding genres
s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


In [ ]:
s.head()

In [ ]:
#Create a new feature by exploding genres
s = df.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)

#Name the new feature as 'genre'
s.name = 'genre'

#Create a new dataframe gen_df which by dropping the old 'genres' feature and adding the new 'genre'.
gen_df = df.drop('genres', axis=1).join(s)

#Print the head of the new gen_df
gen_df.head(10)

## The build_chart function

In [ ]:
def build_chart(genre,low_time,high_time,low_year,high_year,gen_df, percentile=0.8):
  
    
    #Define a new movies variable to store the preferred movies. Copy the contents of gen_df to movies
    movies = gen_df.copy()
    
    #Filter based on the condition
    movies = movies[(movies['genre'] == genre) & 
                    (movies['runtime'] >= low_time) & 
                    (movies['runtime'] <= high_time) & 
                    (movies['year'] >= low_year) & 
                    (movies['year'] <= high_year)]
    
    #Compute the values of C and m for the filtered movies
    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    
    #Only consider movies that have higher than m votes. Save this in a new dataframe q_movies
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    
    #Calculate score using the IMDB formula
    q_movies['score'] = q_movies.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) 
                                       + (m/(m+x['vote_count']) * C)
                                       ,axis=1)

    #Sort movies in descending order of their scores
    q_movies = q_movies.sort_values('score', ascending=False)
    
    return q_movies

In [ ]:
df.head()

In [ ]:
## Convert the cleaned (non-exploded) dataframe df into a CSV file and save it in the data folder
#Set parameter index to False as the index of the DataFrame has no inherent meaning.
df.to_csv('RecomSystm/data/metadata_clean.csv', index=False)

# Content Based

In [ ]:
#Import data from the clean file 
df = pd.read_csv('RecomSystm/data/metadata_clean.csv')

#Print the head of the cleaned DataFrame
df.head()

In [ ]:
# Function to sanitize data to prevent ambiguity. 
# Removes spaces and converts to lowercase
def sanitize(x):
    if isinstance(x, list):
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        

In [ ]:

#Apply the generate_list function title
for feature in ['title']:
    df[feature] = df[feature].apply(sanitize)

In [ ]:
df.shape

In [ ]:
#Import the original file
orig_df = pd.read_csv('RecomSystm/data/movies_metadata.csv', low_memory=False)

#Add the useful features into the cleaned dataframe
df['overview'], df['id'] = orig_df['overview'], orig_df['id']

df.head()

In [ ]:
#Convert the cleaned (non-exploded) dataframe df into a CSV file and save it in the small_dataset folder
#Remove some data(35,467) from it.
#Set parameter index to False as the index of the DataFrame has no inherent meaning.
df.to_csv('small_dataset/metadata_clean.csv', index=False)

In [ ]:
#Import data from the clean file 
df = pd.read_csv('small_dataset/metadata_clean.csv')

#Print the head of the cleaned DataFrame
df = df.head(200)

In [ ]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['overview'] = df['overview'].fillna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(df['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

In [ ]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
#Construct a reverse mapping of indices and movie titles, and drop duplicate titles, if any
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [ ]:
# Function that takes in movie title as input and gives recommendations 
def content_recommender(title, cosine_sim=cosine_sim, df=df, indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [ ]:
#Get recommendations for The Lion King
content_recommender('The Lion King')

# Metadata-based recommender

## Load Data


In [ ]:
# Load the keywords and credits files
cred_df = pd.read_csv('credits.csv')
key_df = pd.read_csv('keywords.csv')

#Print the head of the credit dataframe
cred_df.head()

In [ ]:
#Print the head of the keywords dataframe
key_df.head()

## Data understanding and pre-processing

In [ ]:
df.head()

In [ ]:
#Convert the IDs of df into int
df['id'] = df['id'].astype('int')

In [ ]:
# Function to convert all non-integer IDs to NaN
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

#Clean the ids of df
df['id'] = df['id'].apply(clean_ids)

#Filter all rows that have a null ID
df = df[df['id'].notnull()]

In [ ]:
# Convert IDs into integer
df['id'] = df['id'].astype('int')
key_df['id'] = key_df['id'].astype('int')
cred_df['id'] = cred_df['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
df = df.merge(cred_df, on='id')
df = df.merge(key_df, on='id')

#Display the head of the merged df
df.head()

In [ ]:
# Convert the stringified objects into the native python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']


In [ ]:
for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [ ]:
#Print the first cast member of the first movie in df
df.iloc[0]['crew'][0]

In [ ]:
# Extract the director's name. If director is not listed, return NaN
def get_director(x):
    for crew_member in x:
        if crew_member['job'] == 'Director':
            return crew_member['name']
    return np.nan

In [ ]:
#Define the new director feature
df['director'] = df['crew'].apply(get_director)

#Print the directors of the first five movies
df['director'].head()

In [ ]:
#Print the cast member of the first movie in df
df.iloc[0]['cast']

In [ ]:
# Returns the list top 3 elements or entire list; whichever is more.
def generate_list(x):
    if isinstance(x, list):
        names = [ele['name'] for ele in x]
        #Check if more than 3 elements exist. If yes, return only first three. 
        #If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [ ]:
#Apply the generate_list function to cast and keywords
df['cast'] = df['cast'].apply(generate_list)
df['keywords'] = df['keywords'].apply(generate_list)

#Only consider a maximum of 3 genres
df['genres'] = df['genres'].apply(lambda x: x[:3])

In [ ]:
df['genres'].head()

In [ ]:
# Print the new features of the first 5 movies along with title
df[['title', 'cast', 'director', 'keywords', 'genres']].head()

In [ ]:
# Function to sanitize data to prevent ambiguity. 
# Removes spaces and converts to lowercase
def sanitize(x):
    if isinstance(x, list):
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
#Apply the generate_list function to cast, keywords, director and genres
for feature in ['title','cast', 'director', 'genres', 'keywords']:
    df[feature] = df[feature].apply(sanitize)

In [ ]:
#Apply the generate_list function to cast, keywords, director and genres
for feature in ['title','cast', 'director', 'genres', 'keywords']:
    df[feature] = df[feature].apply(sanitize)

In [ ]:
# Print the new features of the first 5 movies along with title
df[['title', 'cast', 'director', 'keywords', 'genres']].head()

In [ ]:
#Function that creates a soup out of the desired metadata
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [ ]:
# Create the new soup feature
df['soup'] = df.apply(create_soup, axis=1)

In [ ]:
#Display the soup of the first movie
df.iloc[0]['soup']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#Define a new CountVectorizer object and create vectors for the soup
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [ ]:
#Import cosine_similarity function
from sklearn.metrics.pairwise import cosine_similarity

#Compute the cosine similarity score (equivalent to dot product for tf-idf vectors)
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Reset index of your df and construct reverse mapping again
df = df.reset_index()
indices2 = pd.Series(df.index, index=df['title'])

In [ ]:
print("Please Input movie, to Watch similar Movies: ")
movie = input()
#Get recommendations for The Lion King
content_recommender(movie, cosine_sim2, df, indices2)

# Collaborative Based

### USER DS

In [ ]:
import pandas as pd
#Load the u.user file into a dataframe
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv('movielens/u.user', sep='|', names=u_cols, encoding='latin-1')

users.head()

### MOVIE DS

In [ ]:
#Load the u.items file into a dataframe
i_cols = ['movie_id', 'title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('movielens/u.item', sep='|', names=i_cols, encoding='latin-1')

movies.head()

In [ ]:
#Remove all information except Movie ID and title
movies = movies[['movie_id', 'title',]]
movies.head()

### Rating DS

In [ ]:
#Load the u.data file into a dataframe
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('movielens/u.data', sep='\t', names=r_cols, encoding='latin-1')

ratings.head()

In [ ]:
#Drop the timestamp column
ratings = ratings.drop('timestamp', axis=1)

In [ ]:
ratings.head()

In [ ]:
ratings.groupby(['movie_id']).user_id.count().head(20)

In [ ]:
print(users.shape)
print(movies.shape)
print(ratings.shape)

In [ ]:
ratings.info()

In [ ]:
ratings.rating.unique()

In [ ]:
#ratings dataset should have movies only which exist in our movies dataset, unless new movies are added to movies dataset
ratings_new = ratings[ratings.movie_id.isin(movies.movie_id)]

In [ ]:
#ratings dataset will have n_users*n_movies entries if every user rated every item, this shows that the dataset is very sparse
n_users = users.shape[0]
n_movies = movies.shape[0]
print(n_users * n_movies)

In [ ]:
print("number of users: " + str(n_users))
print("number of movies: " + str(n_movies))

In [ ]:
#Sparsity of dataset in %
sparsity=1.0-len(ratings_new)/float(n_users*n_movies)
print('The sparsity level of Movies Crossing dataset is ' +  str(sparsity*100) + ' %')

In [ ]:
#Hence segragating implicit and explict ratings datasets
ratings_explicit = ratings_new[ratings_new.rating != 0]
ratings_implicit = ratings_new[ratings_new.rating == 0]

In [ ]:
#checking shapes
print(ratings_new.shape)
print(ratings_explicit.shape)
print(ratings_implicit.shape)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#plotting count of movieRating
sns.countplot(data=ratings_explicit , x='rating')
plt.show()
#It can be seen that higher ratings are more common amongst users and rating 4 has been rated highest number of times

In [ ]:
#Similarly segregating users who have given explicit ratings from 1-5 and those whose implicit behavior was tracked
users_exp_ratings = users[users.user_id.isin(ratings_explicit.user_id)]
users_imp_ratings = users[users.user_id.isin(ratings_implicit.user_id)]

In [ ]:
#checking shapes
print(users.shape)
print(users_exp_ratings.shape)
print(users_imp_ratings.shape)

**Collaborative Filtering Based Recommendation Systems**

In [ ]:
#To cope up with computing power I have and to reduce the dataset size, I am considering users who have rated atleast 100 movies
#and movies which have atleast 100 ratings
counts1 = ratings_explicit['user_id'].value_counts()
ratings_explicit = ratings_explicit[ratings_explicit['user_id'].isin(counts1[counts1 >= 100].index)]
counts = ratings_explicit['rating'].value_counts()
ratings_explicit = ratings_explicit[ratings_explicit['rating'].isin(counts[counts >= 100].index)]

In [ ]:
#Generating ratings matrix from explicit ratings table
ratings_matrix = ratings_explicit.pivot(index='user_id', columns='movie_id', values='rating')
user_id = ratings_matrix.index
movie_id = ratings_matrix.columns
print(ratings_matrix.shape)
ratings_matrix.head()
#Notice that most of the values are NaN (undefined) implying absence of ratings


In [ ]:
n_users = ratings_matrix.shape[0] #considering only those users who gave explicit ratings
n_movies = ratings_matrix.shape[1]
print(n_users, n_movies)

In [ ]:
import numpy as np

#since NaNs cannot be handled by training algorithms, replacing these by 0, which indicates absence of ratings
#setting data type

ratings_matrix.fillna(0, inplace = True)
ratings_matrix = ratings_matrix.astype(np.int32)

In [ ]:
#checking first few rows
ratings_matrix.head()

In [ ]:
# rechecking the sparsity
sparsity=1.0-len(ratings_explicit)/float(users_exp_ratings.shape[0]*n_movies)
print('The sparsity level of Movies Crossing dataset is ' +  str(sparsity*100) + ' %')

**Training our recommendation system**

In [ ]:
## setting global variables
global metric,k
k=10
metric='cosine'

## User-based Recommendation System

In [ ]:
from sklearn.neighbors import NearestNeighbors

#This function finds k similar users given the user_id and ratings matrix 
#These similarities are same as obtained via using pairwise_distances
def findksimilarusers(user_id, ratings, metric = metric, k=k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
    model_knn.fit(ratings)
    loc = ratings.index.get_loc(user_id)
   # print(loc)
    #print("hi ",ratings.iloc[loc, :].values.reshape(1, -1))
    distances, indices = model_knn.kneighbors(ratings.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
                    
    return similarities,indices

In [ ]:
similarities, indices=findksimilarusers(5, ratings_matrix,metric, k) #similar users based on cosine similarity
similarities

In [ ]:
indices

In [ ]:
item_loc = ratings_matrix.index.get_loc(5)
item_loc


In [ ]:
ratings_matrix.index[0:100]

In [ ]:
ratings_matrix.index[100:200]

In [ ]:
ratings_matrix.index[200:300]

In [ ]:
ratings_matrix.index[300:364]

In [ ]:
# This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, ratings, metric = metric, k=k):
    prediction=0
    user_loc = ratings.index.get_loc(user_id)
    item_loc = ratings.columns.get_loc(item_id)
    similarities, indices=findksimilarusers(user_id, ratings,metric, k) #similar users based on cosine similarity
    mean_rating = ratings.iloc[user_loc,:].mean() #to adjust for zero based indexing
    sum_wt = np.sum(similarities)-1
    #print(sum_wt)
    product=1
    wtd_sum = 0 
    
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] == user_loc:
            continue;                                                                   
        else: 
         #   print("hi ",ratings.iloc[indices.flatten()[i],item_loc])
            ratings_diff = ratings.iloc[indices.flatten()[i],item_loc]-np.mean(ratings.iloc[indices.flatten()[i],:])
         #   print("rating ",ratings_diff)
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product

    
    prediction = int(round(mean_rating + (wtd_sum/sum_wt)))
    #print(prediction)

    #in case of very sparse datasets, using correlation metric for collaborative based approach may give negative ratings
    #which are handled here as below
    if prediction <= 0:
        prediction = 1   
    elif prediction >5:
        prediction = 5
    
    print('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [ ]:
predict_userbased(1,3,ratings_matrix);

## Item-based Recommendation SystemS

In [ ]:
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [ ]:
#This function finds k similar items given the item_id and ratings matrix

def findksimilaritems(item_id, ratings, metric=metric, k=k):
    similarities=[]
    indices=[]
    ratings=ratings.T
    loc = ratings.index.get_loc(item_id)
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute')
    model_knn.fit(ratings)
    
    distances, indices = model_knn.kneighbors(ratings.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()

    return similarities,indices

In [ ]:
#This function predicts the rating for specified user-item combination based on item-based approach
def predict_itembased(user_id, item_id, ratings, metric = metric, k=k):
    prediction= wtd_sum =0
    user_loc = ratings.index.get_loc(user_id)
    item_loc = ratings.columns.get_loc(item_id)
    similarities, indices=findksimilaritems(item_id, ratings,metric, k) #similar users based on correlation coefficients
    sum_wt = np.sum(similarities)-1
    product=1
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] == item_loc:
            continue;
        else:
            product = ratings.iloc[user_loc,indices.flatten()[i]] * (similarities[i])
            wtd_sum = wtd_sum + product                              
    prediction = int(round(wtd_sum/sum_wt))
    
    #in case of very sparse datasets, using correlation metric for collaborative based approach may give negative ratings
    #which are handled here as below //code has been validated without the code snippet below, below snippet is to avoid negative
    #predictions which might arise in case of very sparse datasets when using correlation metric
    if prediction <= 0:
        prediction = 1   
    elif prediction >5:
        prediction = 5
    
    print('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction) )     
    
    return prediction

In [ ]:
predict_itembased(15, 2 ,ratings_matrix, 'correlation')

In [ ]:
#This function utilizes above functions to recommend items for item/user based approach and cosine/correlation. 
#Recommendations are made if the predicted rating for an item is >= to 6,and the items have not been rated already
def recommendItem(user_id, ratings, metric=metric):    
    if (user_id not in ratings.index.values) or type(user_id) is not int:
        print("User id should be a valid integer from this list :\n\n {} ".format(re.sub('[\[\]]', '', np.array_str(ratings_matrix.index.values))))
    else:  
        ids = ['Item-based (correlation)','Item-based (cosine)','User-based (correlation)','User-based (cosine)']
        select = widgets.Dropdown(options=ids, value=ids[0],description='Select approach', width='1000px')

        
        def on_change(change):
           # clear_output(wait=True)
            prediction = []
            df= []     

            if change['type'] == 'change' and change['name'] == 'value':            
                if (select.value == 'Item-based (correlation)') | (select.value == 'User-based (correlation)') :
                    metric = 'correlation'
                else:                       
                    metric = 'cosine'  

                with suppress_stdout():
                    if (select.value == 'Item-based (correlation)') | (select.value == 'Item-based (cosine)'):
                        for i in range(ratings.shape[1]):
                            if (ratings[(ratings.columns[i])][user_id] !=0): #not rated already
                                prediction.append(predict_itembased(user_id, (ratings.columns[i]) ,ratings, metric))
                            else:
                                prediction.append(-1) #for already rated items
                    else:
                        for i in range(ratings.shape[1]):
                            if (ratings[(ratings.columns[i])][user_id] !=0): #not rated already
                                prediction.append(predict_userbased(user_id, (ratings.columns[i]) ,ratings, metric))
                            else:                    
                                prediction.append(-1) #for already rated items
                                
                                
                prediction = pd.Series(prediction)
            #    print(prediction)

                prediction = prediction.sort_values(ascending=False)
                recommended = prediction[:10]
              # print(recommended)
                global df1

                print("As per {0} approach....Following books are recommended...".format(select.value))
                for i in range(len(recommended)):
                    df.append(movies.title[recommended.index[i]])
                
                df1= pd.DataFrame(data=df,columns =['title'])
                print(df1)
               
                
        select.observe(on_change)
        display(select)

In [ ]:
p=recommendItem(10,ratings_matrix)


# GUI

In [ ]:
from tkinter import *
from pandastable import Table, TableModel

#launch the app

def Ranking():
    top = Toplevel() 
    top.title('Ranking Based RS') 
    top.geometry('390x300+530+290')
    df = ranking.head(10)
    table = pt = Table(top, dataframe=df, showtoolbar=True, showstatusbar=True)
    pt.show()
    top.mainloop()


def popular():
    top = Toplevel() 
    top.title('popularity Based RS') 
    top.geometry('390x300+530+290')
    df1 = popular1.head(10)
    table = pt = Table(top, dataframe=df1, showtoolbar=True, showstatusbar=True)
    pt.show()
    top.mainloop()
    


def imdb_ranking():
    top = Toplevel() 
    top.title('IMDB RS') 
    top.geometry('390x300+530+290')
    df1 = imdb_movie.head(10)
    table = pt = Table(top, dataframe=df1, showtoolbar=True, showstatusbar=True)
    pt.show()
    top.mainloop()


def KBB(genre,low_time,high_time,low_year,high_year):
    top = Toplevel() 
    top.title('Knowledge Based Recommendations') 
    top.geometry('390x300+530+290')
    
    x=cb1.get()
    x=x.split('-')
    low_year = int(x[0])
    high_year = int(x[1])


    genre = genre.get()
    low_time=int(low_time.get())
    high_time=int(high_time.get())
    low_year = int(low_year)
    high_year = int(high_year)
    KBS = build_chart(genre,low_time,high_time,low_year,high_year,gen_df).head(10)  
    del KBS['runtime']
    del KBS['vote_count']
    del KBS['year']
    del KBS['score']
    del KBS['vote_average']
    del KBS['genre']
    df=KBS
    print(genre,low_time,high_time,low_year,high_year,df)
    table = pt = Table(top, dataframe=df, showtoolbar=True, showstatusbar=True)
    pt.show()
    top.mainloop()

    

def content(movie):
    top = Toplevel() 
    top.title('Content Based Recommendations') 
    top.geometry('390x300+530+290')

    if(dd.get()=='plot-based'):
        #Get recommendations for The Lion King
        movie=movie.get()
        print(movie)
        s=content_recommender(movie)
        df1=pd.DataFrame(s, columns=['title'])

    else:
        movie=movie.get()
        print(movie)
        s=content_recommender(movie, cosine_sim2, df, indices2)
        df1=pd.DataFrame(s, columns=['title'])
    table = pt = Table(top, dataframe=df1, showtoolbar=True, showstatusbar=True)
    pt.show()
    top.mainloop()

    

def CR():
    top = Toplevel() 
    top.title('Collaborative Filtering Recommendations') 
    top.geometry('390x300+530+290')
    table = pt = Table(top, dataframe=df1, showtoolbar=True, showstatusbar=True)

    if(dd1.get()=='Item-based (correlation)'):
        #Get recommendations for The Lion King
        s=df1
        df=pd.DataFrame(s, columns=['title'])
 
    elif(dd1.get()=='Item-based (cosine)'):
        s=df1
        df=pd.DataFrame(s, columns=['title'])
    
    elif(dd1.get()=='User-based (correlation)'):
        s=df1
        df=pd.DataFrame(s, columns=['title'])
        
    elif(dd1.get()=='User-based (cosine)'):
        s=df1
        df=pd.DataFrame(s, columns=['title'])
        
    table = pt = Table(top, dataframe=df, showtoolbar=True, showstatusbar=True)
    pt.show()
    top.mainloop()

In [ ]:
from tkinter import *
from tkinter import ttk

window = Tk()
window.title("Recommender System")
window.geometry('1400x790')
global s
s=0
#window.configure(bg='red')

tab_control = ttk.Notebook(window)

tab1 = ttk.Frame(tab_control)
tab2 = ttk.Frame(tab_control)
tab3 = ttk.Frame(tab_control)
tab4 = ttk.Frame(tab_control)

tab_control.add(tab1, text='Popularity Based')
tab_control.add(tab2, text='Knowledge Based')
tab_control.add(tab3, text='Content Based')
tab_control.add(tab4, text='Collaborative Filtering')

#tab1
lbl = Label(tab1,padx=500,pady=20, text='Simple Recommendation System',font=("Arial Bold", 20))
lbl.grid(columnspan=4, row=0)
   
button = Button(tab1,text='Based on Popularity', command=popular,pady=7,width=40, font=("Times New Roman bold", 16), bg='light grey', relief = RAISED, bd=3,height = 1,padx=3)
button.grid(column=1, row=2,padx=(20, 20),pady=(10,10))


   
button1 = Button(tab1,text='Based on Ranking', command=Ranking,pady=7,width=40, font=("Times New Roman bold", 16), bg='light grey', relief = RAISED, bd=3,height = 1,padx=3)
button1.grid(column=2, row=2,padx=(20, 20),pady=(10,10))


button2 = Button(tab1,text='IMDB Ranking', command=imdb_ranking,pady=7,width=40, font=("Times New Roman bold", 16), bg='light grey', relief = RAISED, bd=3,height = 1,padx=3)
button2.grid(columnspan=4, row=7,padx=(20, 20),pady=(10,10))

# tab2
lbl = Label(tab2,padx=500,pady=20, text='Knowledge Based Recommender System',font=("Arial Bold", 20))
lbl.grid(columnspan=4, row=0)

lbl2 = Label(tab2,padx=30,pady=20, text='Please Enter Minimum Duration of MOvie (in minutes)',font=("Arial Bold", 12))
lbl2.grid(column=0, row=1,padx=(50, 30),pady=(10,10))


low_time = Entry(tab2,width=40)
low_time.grid(column=0, row=2,padx=(40, 20),pady=(10,10))

lbl2 = Label(tab2,padx=30,pady=20, text='Please Enter Maximum Duration of MOvie (in minutes)',font=("Arial Bold", 12))
lbl2.grid(column=1, row=1)

#high_time = Spinbox(tab2, from_=0, to=100, width=40)
#high_time.grid(column=1, row=2)
#high_time.focus_set()


high_time = Entry(tab2,width=40)
high_time.grid(column=1, row=2,padx=(40, 20),pady=(10,10))

lbl3 = Label(tab2,padx=30,pady=30, text='Please Enter Genre of Movie',font=("Arial Bold", 12))
lbl3.grid(column=0, row=5)


Genre=["animation","comedy","family","adventure","fantasy","romance","drama","crime","action","thriller","horror","science fiction","mystery","tv movie"]

genre=ttk.Combobox(tab2,values=Genre,width=40)
genre.grid(column=0, row=6)
genre.current(0)

lbl4 = Label(tab2,padx=30,pady=30, text='Please Enter Timeline of Movie',font=("Arial Bold", 12))
lbl4.grid(column=1, row=5)

timeline=["1974-1979","1980-1985","1986-1991","1992-1997","1998-2003","2002-2007","200"]
cb1=ttk.Combobox(tab2,values=timeline,width=40)
cb1.grid(column=1, row=6)
cb1.current(0)

x=cb1.get()
x=x.split('-')
low_year = int(x[0])
high_year = int(x[1])

    
#b=Button(tab2,text="Click Here",command=func)
#b.grid(column=0, row=5)

l2=Label(tab2,text=low_time.get())
l2.grid(column=0, row=7)

#Button(tab4, text="Choose File", bg='blue', fg='white',width=40,font=("Arial Bold", 10), activebackground='black', activeforeground='grey',  ).grid(column=1, row=1)
button = Button(tab2, text="Submit",command= lambda: KBB(genre,low_time,high_time,low_year,high_year),pady=7,width=40, font=("Times New Roman bold", 16), bg='light grey', relief = RAISED, bd=3,height = 1)
button.grid(columnspan=3, row=8,padx=(10, 10),pady=(150,70))

#tab3
lbl = Label(tab3,padx=500,pady=20, text='Content Based Recommender System',font=("Arial Bold", 20))
lbl.grid(columnspan=4, row=0)

lbl3 = Label(tab3, text='Please Enter Type of Content Recommender',font=("Arial Bold", 12))
lbl3.grid(column=0, row=2,padx=(200, 10),pady=(100,100))


dd=["plot-based","meta-based"]
dd=ttk.Combobox(tab3,values=dd,width=50)
dd.grid(column=1, row=2,padx=(10, 10),pady=(100,100))
dd.current(0)

lbl3 = Label(tab3, text='Please Enter Movie Name',font=("Arial Bold", 12))
lbl3.grid(column=0, row=3,padx=(200, 10),pady=(10,10))


movie = Entry(tab3,width=50)
movie.grid(column=1, row=3,padx=(10, 10),pady=(10,10))

button = Button(tab3, text="Submit",command= lambda: content(movie),pady=7,width=40, font=("Times New Roman bold", 16), bg='light grey', relief = RAISED, bd=3,height = 1)
button.grid(columnspan=3, row=4,padx=(10, 10),pady=(100,70))
#tab4

lbl = Label(tab4,padx=500,pady=20, text='Collaborative Filtering Based Recommender System',font=("Arial Bold", 20))
lbl.grid(columnspan=4, row=0)

lbl3 = Label(tab4, text='Please Enter Type of Content Recommender',font=("Arial Bold", 12))
lbl3.grid(column=0, row=2,padx=(200, 10),pady=(100,100))


dd1=['Item-based (correlation)','Item-based (cosine)','User-based (correlation)','User-based (cosine)']
dd1=ttk.Combobox(tab4,values=dd1,width=50)
dd1.grid(column=1, row=2,padx=(10, 10),pady=(100,100))
dd1.current(0)

button = Button(tab4, text="Submit",command= lambda: CR(),pady=7,width=40, font=("Times New Roman bold", 16), bg='light grey', relief = RAISED, bd=3,height = 1)
button.grid(columnspan=3, row=4,padx=(10, 10),pady=(100,70))


tab_control.pack(expand=5, fill='both')
window.mainloop()